In [1]:
import os
import pandas as pd
import numpy as np
import requests
import pandas as pd
from msal import ConfidentialClientApplication

# 1. 인증 설정
def get_access_token(client_id, client_secret, tenant_id):
    """
    Microsoft Graph API에 접근하기 위한 OAuth2 인증 토큰을 받음.
    """
    authority = f"https://login.microsoftonline.com/{tenant_id}"
    app = ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
    result = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    
    if "access_token" in result:
        return result["access_token"]
    else:
        raise Exception("The token can't be taken. Please check again.")

# 2. 메일 전송 함수
def send_email(access_token, recipient_email, recipient_name, title, body, cc_emails=None):
    """
    Microsoft Graph API를 사용해 Outlook에서 이메일을 보냄.
    """
    
    cc_recipients = [{"emailAddress": {"address": email}} for email in cc_emails] if cc_emails else []
    
    url = "https://graph.microsoft.com/v1.0/me/sendMail"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    email_data = {
        "message": {
            "subject": title,
            "body": {
                "contentType": "Text",
                "content": body.format(name=recipient_name)  # 템플릿 형식으로 이름 포함
            },
            "toRecipients": [
                {
                    "emailAddress": {
                        "address": recipient_email
                    }
                }
            ],
            # 참조 대상이 있을 경우 추가
            "ccRecipients": cc_recipients
        }
    }
    response = requests.post(url, headers=headers, json=email_data)
    if response.status_code == 202:
        print(f"{recipient_name}에게 이메일이 성공적으로 전송되었습니다.")
    else:
        print(f"이메일 전송 실패: {response.json()}")

# 3. 엑셀 데이터에서 정보 추출하여 메일 보내기
def send_emails_from_excel(client_id, client_secret, tenant_id, excel_file_path):
    """
    엑셀 파일을 읽고 각 수신자에게 자동으로 메일을 보냄.
    """
    # 엑셀 파일 읽기
    df = pd.read_excel(excel_file_path)
    
    # API 액세스 토큰 가져오기
    access_token = get_access_token(client_id, client_secret, tenant_id)
    
    # 엑셀의 각 행에서 정보 추출 후 메일 전송
    for _, row in df.iterrows():
        recipient_email = row["recipient_email"]
        recipient_name = row["recipient_name"]
        title = "행사 안내"  # 예시 제목
        body = f"안녕하세요, {recipient_name}님. 이번 행사에 초대합니다!"  # 본문 템플릿
        
        send_email(access_token, recipient_email, recipient_name, title, body)

def read_xsl_files(directory):
    data_frames = {}
    print("\nReading Excel files:")
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            print(f"  Reading: {filename}")
            df = pd.read_csv(os.path.join(directory, filename))
            parts = filename.split('_')
            participant = parts[-2] if len(parts) > 2 else 'unknown'
            difficulty = parts[-1].split('.')[0] if len(parts) > 1 else 'unknown'
            key = (participant, difficulty)
            data_frames[key] = df
    print(f"Total files read: {len(data_frames)}")
    return data_frames

def calculate_dt(df):
    dt_result = {}
    
    for (participant, difficulty), df in df.items():
        time = df.pop('UNIX Time')
        %matplotlib inline
        df.plot()
        key = (participant, difficulty)
        dt = pd.DataFrame()
        for i in range(len(df) - 1):
            dt = pd.concat([dt, ((df.iloc[i + 1] - df.iloc[i]) / 0.2).to_frame().T], ignore_index=True)
        dt.plot()
        dt.insert(0, 'UNIX Time', time)
        print(dt)
        dt.to_csv(f"C:/Users/USER/Downloads/pilotdata_performance/differential_{participant}_{difficulty}.csv", index=False)
        dt_result[key] = dt
    return dt_result

def main():
        # 엑셀 파일 경로와 Azure 인증 정보 입력 후 호출
    client_id = "Your_Client_ID"
    client_secret = "Your_Client_Secret"
    tenant_id = "Your_Tenant_ID"
    excel_file_path = r'C:\Users\USER\Downloads\pilotdata_performance'

    send_emails_from_excel(client_id, client_secret, tenant_id, excel_file_path)
    directory = r'C:\Users\USER\Downloads\pilotdata_performance'
    data_frames = read_xsl_files(directory)
    
if __name__ == "__main__":
    main()

KeyboardInterrupt: 